In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/bigdata

/content/drive/MyDrive/Colab Notebooks/bigdata


In [4]:
df = pd.read_csv('data.csv')
df.shape

(200092, 331)

In [5]:
df.columns

Index(['SERIAL_NUM', 'CUST_LOCATION', 'AGE', 'GENDER', 'JOB', 'BRANCH_DIST',
       'AMT_EXCHANGE_A_M1', 'AMT_EXCHANGE_A_M2', 'AMT_EXCHANGE_A_M3',
       'AMT_EXCHANGE_A_M4',
       ...
       'AUM_M4', 'AUM_M5', 'AUM_M6', 'AUM_M7', 'AUM_M8', 'AUM_M9', 'AUM_M10',
       'AUM_M11', 'AUM_M12', 'AUM_M13'],
      dtype='object', length=331)

## 取出基本資料column備用

In [6]:
col_info = ['CUST_LOCATION', 'AGE', 'GENDER', 'JOB', 'BRANCH_DIST']

## 取出變動資訊並照日期排序，在最後一個columns加上'AUM_M13'

In [7]:
col_train = ['AMT_EXCHANGE_A_M' + str(i) for i in np.arange(1, 13)] + ['AMT_EXCHANGE_B_M' + str(i) for i in np.arange(1, 13)] + \
            ['AMT_EXCHANGE_C_M' + str(i) for i in np.arange(1, 13)] + ['AMT_INVST_M' + str(i) for i in np.arange(1, 13)] + \
            ['AMT_LOAN_D_M' + str(i) for i in np.arange(1, 13)] +  ['AMT_LOAN_E_M' + str(i) for i in np.arange(1, 13)] + \
            ['AMT_LOAN_F_M' + str(i) for i in np.arange(1, 13)] + ['AMT_INTERACT_G_M' + str(i) for i in np.arange(1, 13)] + \
            ['AMT_INTERACT_H_M' + str(i) for i in np.arange(1, 13)] + ['AUM_I_M' + str(i) for i in np.arange(1, 13)] +\
            ['AUM_J_M' + str(i) for i in np.arange(1, 13)] + ['AUM_K_M' + str(i) for i in np.arange(1, 13)] + \
            ['AUM_L_M' + str(i) for i in np.arange(1, 13)] +  ['AUM_M_M' + str(i) for i in np.arange(1, 13)] + \
            ['AUM_M' + str(i) for i in np.arange(1, 13)]
m1 = [True if i.endswith('_M1') else False for i in col_train]
m2 = [True if i.endswith('_M2') else False for i in col_train]
m3 = [True if i.endswith('_M3') else False for i in col_train]
m4 = [True if i.endswith('_M4') else False for i in col_train]
m5 = [True if i.endswith('_M5') else False for i in col_train]
m6 = [True if i.endswith('_M6') else False for i in col_train]
m7 = [True if i.endswith('_M7') else False for i in col_train]
m8 = [True if i.endswith('_M8') else False for i in col_train]
m9 = [True if i.endswith('_M9') else False for i in col_train]
m10 = [True if i.endswith('_M10') else False for i in col_train]
m11 = [True if i.endswith('_M11') else False for i in col_train]
m12 = [True if i.endswith('_M12') else False for i in col_train]

len(col_train) # 180 = 12*15

180

In [8]:
col_train_sorted = []
for m in [m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11, m12]:
    col_train_sorted.extend(list(d for d, s in zip(col_train, m) if s))
    
len(col_train_sorted)
# col_train_sorted[-15:]
df_sort = df[col_train_sorted + ['AUM_M13']]
df_sort.shape
# df_sort.columns

(200092, 181)

## min max scaler

In [9]:
from sklearn import preprocessing

df_sort_nor = pd.DataFrame()
min_max_scaler = preprocessing.MinMaxScaler()
cols = list(df_sort.columns)
for col in cols:
  df_sort_nor[col] = min_max_scaler.fit_transform(df_sort[col].values.reshape(-1, 1)).flatten()

df_sort_nor.head()

,AMT_EXCHANGE_A_M1,AMT_EXCHANGE_B_M1,AMT_EXCHANGE_C_M1,AMT_INVST_M1,AMT_LOAN_D_M1,AMT_LOAN_E_M1,AMT_LOAN_F_M1,AMT_INTERACT_G_M1,AMT_INTERACT_H_M1,AUM_I_M1,AUM_J_M1,AUM_K_M1,AUM_L_M1,AUM_M_M1,AUM_M1,AMT_EXCHANGE_A_M2,AMT_EXCHANGE_B_M2,AMT_EXCHANGE_C_M2,AMT_INVST_M2,AMT_LOAN_D_M2,AMT_LOAN_E_M2,AMT_LOAN_F_M2,AMT_INTERACT_G_M2,AMT_INTERACT_H_M2,AUM_I_M2,AUM_J_M2,AUM_K_M2,AUM_L_M2,AUM_M_M2,AUM_M2,AMT_EXCHANGE_A_M3,AMT_EXCHANGE_B_M3,AMT_EXCHANGE_C_M3,AMT_INVST_M3,AMT_LOAN_D_M3,AMT_LOAN_E_M3,AMT_LOAN_F_M3,AMT_INTERACT_G_M3,AMT_INTERACT_H_M3,AUM_I_M3,...,AMT_LOAN_F_M10,AMT_INTERACT_G_M10,AMT_INTERACT_H_M10,AUM_I_M10,AUM_J_M10,AUM_K_M10,AUM_L_M10,AUM_M_M10,AUM_M10,AMT_EXCHANGE_A_M11,AMT_EXCHANGE_B_M11,AMT_EXCHANGE_C_M11,AMT_INVST_M11,AMT_LOAN_D_M11,AMT_LOAN_E_M11,AMT_LOAN_F_M11,AMT_INTERACT_G_M11,AMT_INTERACT_H_M11,AUM_I_M11,AUM_J_M11,AUM_K_M11,AUM_L_M11,AUM_M_M11,AUM_M11,AMT_EXCHANGE_A_M12,AMT_EXCHANGE_B_M12,AMT_EXCHANGE_C_M12,AMT_INVST_M12,AMT_LOAN_D_M12,AMT_LOAN_E_M12,AMT_LOAN_F_M12,AMT_INTERACT_G_M12,AMT_INTERACT_H_M12,AUM_I_M12,AUM_J_M12,AUM_K_M12,AUM_L_M12,AUM_M_M12,AUM_M12,AUM_M13
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006129,0.0,0.0,0.0,0.0,0.006129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001247,0.0,0.0,0.0,0.0,0.001247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001287,...,0.0,0.0,0.0,0.001249,0.0,0.0,0.0,0.0,0.001249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001242,0.0,0.0,0.0,0.0,0.001234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001239,0.0,0.0,0.0,0.0,0.001239,0.001240
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.173870,0.0,0.0,0.0,0.0,0.173870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035363,0.0,0.0,0.0,0.0,0.035363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036499,...,0.0,0.0,0.0,0.035429,0.0,0.0,0.0,0.0,0.035429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035239,0.0,0.0,0.0,0.0,0.034994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035145,0.0,0.0,0.0,0.0,0.035145,0.035183
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.170239,0.0,0.0,0.0,0.0,0.170239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040613,0.0,0.0,0.0,0.0,0.040613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045928,...,0.0,0.0,0.0,0.108094,0.0,0.0,0.0,0.0,0.108094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.107513,0.0,0.0,0.0,0.0,0.106765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.107200,0.0,0.0,0.0,0.0,0.107200,0.107266
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019406,0.0,0.0,0.0,0.0,0.019406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003947,0.0,0.0,0.0,0.0,0.003947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004074,...,0.0,0.0,0.0,0.003954,0.0,0.0,0.0,0.0,0.003954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003933,0.0,0.0,0.0,0.0,0.003906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003923,0.0,0.0,0.0,0.0,0.003923,0.003927
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.216141,0.0,0.0,0.0,0.0,0.216141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043948,0.0,0.0,0.0,0.0,0.043948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045341,...,0.0,0.0,0.0,0.045006,0.0,0.0,0.0,0.0,0.045006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044765,0.0,0.0,0.0,0.0,0.044453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044644,0.0,0.0,0.0,0.0,0.044644,0.044693


## train/test split 

In [10]:
from sklearn.model_selection import train_test_split

X = df_sort_nor.drop('AUM_M13', axis = 1)
Y = df_sort_nor['AUM_M13']

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size = 0.3, random_state = 1, shuffle = True)
train_index = train_x.index
print(f'train shape = {train_x.shape}\ntest shape = {test_x.shape}')

train shape = (140064, 180)
test shape = (60028, 180)


## 用'AUM_M12'的四分位數將train data分成四份

In [11]:
q1, q2, q3 = np.percentile(train_x['AUM_M12'], [25, 50, 75])
q1, q2, q3

(0.001363759660697001, 0.0077195961164973695, 0.041651220954070464)

In [12]:
train_x_1 = train_x[train_x['AUM_M12'] < q1]
train_y_1 = train_y[train_x_1.index]

train_x_2 = train_x[(train_x['AUM_M12'] > q1) & (train_x['AUM_M12'] < q2)]
train_y_2 = train_y[train_x_2.index]

train_x_3 = train_x[(train_x['AUM_M12'] > q2) & (train_x['AUM_M12'] < q3)]
train_y_3 = train_y[train_x_3.index]

train_x_4 = train_x[train_x['AUM_M12'] > q3]
train_y_4 = train_y[train_x_4.index]

print(f'train_x_1 shape = {train_x_1.shape}\ntrain_x_1 shape = {train_x_2.shape}\ntrain_x_1 shape = {train_x_3.shape}\ntrain_x_1 shape = {train_x_4.shape}')

train_x_1 shape = (35016, 180)
train_x_1 shape = (35016, 180)
train_x_1 shape = (35016, 180)
train_x_1 shape = (35016, 180)


## reshape for LSTM

In [13]:
# train_x_1_lstm = np.array(train_x_1).reshape(train_x_1.shape[0], 12, 15)
# train_x_2_lstm = np.array(train_x_2).reshape(train_x_2.shape[0], 12, 15)
# train_x_3_lstm = np.array(train_x_3).reshape(train_x_3.shape[0], 12, 15)
# train_x_4_lstm = np.array(train_x_4).reshape(train_x_4.shape[0], 12, 15)

# train_y_1_lstm = np.array(train_y_1)
# train_y_2_lstm = np.array(train_y_2)
# train_y_3_lstm = np.array(train_y_3)
# train_y_4_lstm = np.array(train_y_4)

# train_x_1_lstm.shape, train_y_1_lstm.shape

## Build model

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, SimpleRNN, BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import plot_model
import time

In [15]:
def model_1():
  model_1 = Sequential()
  model_1.add(LSTM(32, input_shape = (12, 15), return_sequences = False))
  model_1.add(Dense(1))
  model_1.compile(loss = 'mean_absolute_error', optimizer = 'adam')
  return model_1

model_1().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                6144      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 6,177
Trainable params: 6,177
Non-trainable params: 0
_________________________________________________________________


In [16]:
def model_2():
  model_2 = Sequential()
  model_2.add(LSTM(32, input_shape = (12, 15), return_sequences = True))
  model_2.add(LSTM(32, input_shape = (12, 15), return_sequences = False))
  model_2.add(Dense(1))

  model_2.compile(loss = 'mean_absolute_error', optimizer = 'adam')
  return model_2
  
model_2().summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 12, 32)            6144      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 14,497
Trainable params: 14,497
Non-trainable params: 0
_________________________________________________________________


In [17]:
def model_3():
  model_3 = Sequential()
  model_3.add(LSTM(32, input_shape = (12, 15), return_sequences = True))
  model_3.add(LSTM(64, input_shape = (12, 15), return_sequences = False))
  model_3.add(Dense(1))

  model_3.compile(loss = 'mean_absolute_error', optimizer = 'adam')
  return model_3

model_3().summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 12, 32)            6144      
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 31,041
Trainable params: 31,041
Non-trainable params: 0
_________________________________________________________________


In [18]:
def model_4():
  model_4 = Sequential()
  model_4.add(LSTM(32, input_shape = (12, 15), return_sequences = True))
  model_4.add(Dropout(0.2))
  model_4.add(LSTM(32, input_shape = (12, 15), return_sequences = False))
  model_4.add(Dense(1))

  model_4.compile(loss = 'mean_absolute_error', optimizer = 'adam')
  return model_4

model_4().summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 12, 32)            6144      
_________________________________________________________________
dropout (Dropout)            (None, 12, 32)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 14,497
Trainable params: 14,497
Non-trainable params: 0
_________________________________________________________________


In [19]:
def model_5():
  model_5 = Sequential()
  model_5.add(LSTM(32, input_shape = (12, 15), return_sequences = True))
  model_5.add(Dropout(0.2))
  model_5.add(LSTM(64, input_shape = (12, 15), return_sequences = False))
  model_5.add(Dense(1))

  model_5.compile(loss = 'mean_absolute_error', optimizer = 'adam')
  return model_5

model_5().summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 12, 32)            6144      
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 32)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 31,041
Trainable params: 31,041
Non-trainable params: 0
_________________________________________________________________


In [20]:
def model_6():
  model_6 = Sequential()
  model_6.add(LSTM(32, input_shape = (12, 15), return_sequences = True))
  model_6.add(LSTM(32, input_shape = (12, 15), return_sequences = False))
  model_6.add(Dropout(0.2))
  model_6.add(Dense(1))

  model_6.compile(loss = 'mean_absolute_error', optimizer = 'adam')
  return model_6

model_6().summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 12, 32)            6144      
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 14,497
Trainable params: 14,497
Non-trainable params: 0
_________________________________________________________________


In [21]:
def model_7():
  model_7 = Sequential()
  model_7.add(LSTM(32, input_shape = (12, 15), return_sequences = True))
  model_7.add(LSTM(64, input_shape = (12, 15), return_sequences = False))
  model_7.add(Dropout(0.2))
  model_7.add(Dense(1))

  model_7.compile(loss = 'mean_absolute_error', optimizer = 'adam')
  return model_7

model_7().summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 12, 32)            6144      
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 31,041
Trainable params: 31,041
Non-trainable params: 0
_________________________________________________________________


In [22]:
def model_8():
  model_8 = Sequential()

  model_8.add(LSTM(64, input_shape = (12, 15), return_sequences = False))
  model_8.add(Dense(1))

  model_8.compile(loss = 'mean_absolute_error', optimizer = 'adam')
  return model_8

model_8().summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 64)                20480     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 20,545
Trainable params: 20,545
Non-trainable params: 0
_________________________________________________________________


In [23]:
def inverse_transform(y):
  min_max_scaler = preprocessing.MinMaxScaler()
  min_max_scaler.fit_transform(df_sort['AUM_M13'].values.reshape(-1, 1))
  inverse_y = min_max_scaler.inverse_transform(y.reshape(-1, 1)).flatten()
  return inverse_y

In [24]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def corss_validate(model_name, train_x_n, train_y_n, fold):
  start = time.time()
  kf = KFold(n_splits = 5, shuffle = False)
  mse = []
  mae = []
  mape = []
  train_x_n_reset = train_x_n.values
  train_y_n_reset = train_y_n.values
  num = 0
  for train_index, val_index in kf.split(train_x_n_reset, train_y_n_reset):
    train_x_n_lstm = np.array(train_x_n_reset[train_index, :]).reshape(train_x_n_reset[train_index, :].shape[0], 12, 15)
    train_y_n_lstm = np.array(train_y_n_reset[train_index])
    val_x_n_lstm = np.array(train_x_n_reset[val_index, :]).reshape(train_x_n_reset[val_index, :].shape[0], 12, 15)
    val_y_n_lstm = np.array(train_y_n_reset[val_index])

    model = model_name()
    model.fit(train_x_n_lstm, train_y_n_lstm, batch_size = 32, epochs = 10, verbose = 0)

    pred_val_y_n_lstm = model.predict(val_x_n_lstm)
    pred_inverse_val_y_n_lstm = inverse_transform(pred_val_y_n_lstm)
    real_inverse_val_y_n_lstm = inverse_transform(val_y_n_lstm)
    real_inverse_val_y_n_lstm += 0.01
    mse.append(mean_squared_error(pred_inverse_val_y_n_lstm, real_inverse_val_y_n_lstm))
    mae.append(mean_absolute_error(pred_inverse_val_y_n_lstm, real_inverse_val_y_n_lstm))
    # return pd.DataFrame({'pred': pred_inverse_val_y_n_lstm, 'real': real_inverse_val_y_n_lstm})
    # break
    mape.append(np.mean(abs(pred_inverse_val_y_n_lstm - real_inverse_val_y_n_lstm)/real_inverse_val_y_n_lstm))
    num += 1
    print(f'{num}/{fold} done.')
  cv_report = pd.DataFrame({'mse': mse, 'mae': mae, 'mape': mape})
  print('mean mse =', np.mean(cv_report.iloc[:, 0]), ', std =', np.std(cv_report.iloc[:, 0]))
  print('mean mae =', np.mean(cv_report.iloc[:, 1]), ', std =', np.std(cv_report.iloc[:, 1]))
  print('mean mape =', np.mean(cv_report.iloc[:, 2]), ', std =', np.std(cv_report.iloc[:, 2]))
  end = time.time()
  print(f'time used = {np.round(end - start, 2)} seconds.')
  return cv_report

## 對train_1 cross validate 8個模型

In [ ]:
cv_report1_1 = corss_validate(model_1, train_x_1, train_y_1, 5)
cv_report1_1

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
time used = 130.88097715377808seconds.


,mse,mae,mape
0,3.825751e+09,9910.959414,11101.033492
1,2.883456e+09,8020.029854,4992.935987
2,6.133449e+09,12297.706167,33298.153798
3,1.955035e+10,10713.913512,6446.268521
4,3.053493e+09,9840.456722,22425.117344


In [ ]:
cv_report1_2 = corss_validate(model_2, train_x_1, train_y_1, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7106699727.351706 , std = 6344035713.246571
mean mae = 9777.684235827164 , std = 1309.251613836599
mean mape = 10721.768790504904 , std = 5569.644314724583
time used = 194.85 seconds.


In [ ]:
cv_report1_3 = corss_validate(model_3, train_x_1, train_y_1, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7110323324.565618 , std = 6336106044.104774
mean mae = 9757.756176892903 , std = 1339.6163555262483
mean mape = 9630.123995395272 , std = 4587.046392673481
time used = 194.56 seconds.


In [ ]:
cv_report1_4 = corss_validate(model_4, train_x_1, train_y_1, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7107676749.251999 , std = 6347424858.058629
mean mae = 9873.169857364706 , std = 1393.712412564431
mean mape = 10221.086784603933 , std = 6144.8066885778935
time used = 201.75 seconds.


In [ ]:
cv_report1_5 = corss_validate(model_5, train_x_1, train_y_1, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7096797213.568514 , std = 6352838580.5985985
mean mae = 9829.841511902583 , std = 948.3688025621086
mean mape = 12502.53562811825 , std = 7578.715283702105
time used = 202.74 seconds.


In [ ]:
cv_report1_6 = corss_validate(model_6, train_x_1, train_y_1, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7097767034.458722 , std = 6361799336.600014
mean mae = 9929.830689329307 , std = 1128.6562749156544
mean mape = 13872.116991400102 , std = 5460.941805277357
time used = 198.99 seconds.


In [ ]:
cv_report1_7 = corss_validate(model_7, train_x_1, train_y_1, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7091994743.136722 , std = 6342664259.354431
mean mae = 9610.47158520182 , std = 913.8448852946912
mean mape = 11897.16542166922 , std = 5411.348132370227
time used = 199.27 seconds.


In [ ]:
cv_report1_8 = corss_validate(model_8, train_x_1, train_y_1, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7108812856.418208 , std = 6295674046.336836
mean mae = 10430.99020901821 , std = 1691.4038744574584
mean mape = 18052.945843447324 , std = 12951.474859620717
time used = 126.86 seconds.


In [ ]:
cv_report1 = pd.DataFrame(columns = ['mse', 'mae', 'mape'], index = ['model_1', 'model_2', 'model_3', 'model_4',  'model_5',  'model_6',  'model_7', 'model_8'])
cv_report1.iloc[0, :] = [np.mean(cv_report1_1['mse']), np.mean(cv_report1_1['mae']), np.mean(cv_report1_1['mape'])]
cv_report1.iloc[1, :] = [np.mean(cv_report1_2['mse']), np.mean(cv_report1_2['mae']), np.mean(cv_report1_2['mape'])]
cv_report1.iloc[2, :] = [np.mean(cv_report1_3['mse']), np.mean(cv_report1_3['mae']), np.mean(cv_report1_3['mape'])]
cv_report1.iloc[3, :] = [np.mean(cv_report1_4['mse']), np.mean(cv_report1_4['mae']), np.mean(cv_report1_4['mape'])]
cv_report1.iloc[4, :] = [np.mean(cv_report1_5['mse']), np.mean(cv_report1_5['mae']), np.mean(cv_report1_5['mape'])]
cv_report1.iloc[5, :] = [np.mean(cv_report1_6['mse']), np.mean(cv_report1_6['mae']), np.mean(cv_report1_6['mape'])]
cv_report1.iloc[6, :] = [np.mean(cv_report1_7['mse']), np.mean(cv_report1_7['mae']), np.mean(cv_report1_7['mape'])]
cv_report1.iloc[7, :] = [np.mean(cv_report1_8['mse']), np.mean(cv_report1_8['mae']), np.mean(cv_report1_8['mape'])]
cv_report1.to_csv('cv_report1.csv', index = False)
cv_report1

,mse,mae,mape
model_1,7.0893e+09,10156.6,15652.7
model_2,7.1067e+09,9777.68,10721.8
model_3,7.11032e+09,9757.76,9630.12
model_4,7.10768e+09,9873.17,10221.1
model_5,7.0968e+09,9829.84,12502.5
model_6,7.09777e+09,9929.83,13872.1
model_7,7.09199e+09,9610.47,11897.2
model_8,7.10881e+09,10431,18052.9


## 對train_2 cross validate 8個模型

In [ ]:
cv_report2_1 = corss_validate(model_1, train_x_2, train_y_2, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7094516514.764813 , std = 2212754670.8814034
mean mae = 18636.817410516527 , std = 785.0489355661045
mean mape = 1296.575521860659 , std = 900.563344384689
time used = 127.84 seconds.


In [ ]:
cv_report2_2 = corss_validate(model_2, train_x_2, train_y_2, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7076200960.944684 , std = 2270438953.4293065
mean mae = 18168.738420836493 , std = 1082.2565775092871
mean mape = 1311.1111163824214 , std = 935.069519304295
time used = 194.59 seconds.


In [ ]:
cv_report2_3 = corss_validate(model_3, train_x_2, train_y_2, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7114722131.116946 , std = 2270945649.983155
mean mae = 18448.325139192573 , std = 973.9831060664633
mean mape = 1337.7208713613304 , std = 972.3475517818043
time used = 195.96 seconds.


In [ ]:
cv_report2_4 = corss_validate(model_4, train_x_2, train_y_2, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7091888303.885447 , std = 2258041368.379658
mean mae = 18525.759359645603 , std = 1068.7149230903071
mean mape = 1303.0409728021882 , std = 983.1487129666733
time used = 201.78 seconds.


In [ ]:
cv_report2_5 = corss_validate(model_5, train_x_2, train_y_2, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7078795700.434525 , std = 2214297224.6225114
mean mae = 18464.14292599707 , std = 1334.2706023935614
mean mape = 1341.530858168042 , std = 951.0190077079341
time used = 201.46 seconds.


In [ ]:
cv_report2_6 = corss_validate(model_6, train_x_2, train_y_2, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7124634055.61775 , std = 2273551273.369031
mean mae = 18251.376261112124 , std = 923.821486200703
mean mape = 1257.7425106899268 , std = 912.597072918647
time used = 198.17 seconds.


In [ ]:
cv_report2_7 = corss_validate(model_7, train_x_2, train_y_2, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7097987733.244741 , std = 2214840702.9220195
mean mae = 18579.0227724144 , std = 1374.6744907852017
mean mape = 1243.36887737533 , std = 813.8693249384385
time used = 198.29 seconds.


In [ ]:
cv_report2_8 = corss_validate(model_8, train_x_2, train_y_2, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 7086012190.180203 , std = 2242032230.386
mean mae = 18236.877254295658 , std = 1212.5775391593627
mean mape = 1396.0545030523404 , std = 972.0588928415691
time used = 127.43 seconds.


In [ ]:
cv_report2 = pd.DataFrame(columns = ['mse', 'mae', 'mape'], index = ['model_1', 'model_2', 'model_3', 'model_4',  'model_5',  'model_6',  'model_7', 'model_8'])
cv_report2.iloc[0, :] = [np.mean(cv_report2_1['mse']), np.mean(cv_report2_1['mae']), np.mean(cv_report2_1['mape'])]
cv_report2.iloc[1, :] = [np.mean(cv_report2_2['mse']), np.mean(cv_report2_2['mae']), np.mean(cv_report2_2['mape'])]
cv_report2.iloc[2, :] = [np.mean(cv_report2_3['mse']), np.mean(cv_report2_3['mae']), np.mean(cv_report2_3['mape'])]
cv_report2.iloc[3, :] = [np.mean(cv_report2_4['mse']), np.mean(cv_report2_4['mae']), np.mean(cv_report2_4['mape'])]
cv_report2.iloc[4, :] = [np.mean(cv_report2_5['mse']), np.mean(cv_report2_5['mae']), np.mean(cv_report2_5['mape'])]
cv_report2.iloc[5, :] = [np.mean(cv_report2_6['mse']), np.mean(cv_report2_6['mae']), np.mean(cv_report2_6['mape'])]
cv_report2.iloc[6, :] = [np.mean(cv_report2_7['mse']), np.mean(cv_report2_7['mae']), np.mean(cv_report2_7['mape'])]
cv_report2.iloc[7, :] = [np.mean(cv_report2_8['mse']), np.mean(cv_report2_8['mae']), np.mean(cv_report2_8['mape'])]
cv_report2.to_csv('cv_report2.csv', index = False)
cv_report2

,mse,mae,mape
model_1,7.09452e+09,18636.8,1296.58
model_2,7.0762e+09,18168.7,1311.11
model_3,7.11472e+09,18448.3,1337.72
model_4,7.09189e+09,18525.8,1303.04
model_5,7.0788e+09,18464.1,1341.53
model_6,7.12463e+09,18251.4,1257.74
model_7,7.09799e+09,18579,1243.37
model_8,7.08601e+09,18236.9,1396.05


## 對train_3 cross validate 8個模型

In [ ]:
cv_report3_1 = corss_validate(model_1, train_x_3, train_y_3, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 26118284671.08397 , std = 5542182932.264875
mean mae = 49693.209851755484 , std = 1982.2519390577797
mean mape = 2400.112801787642 , std = 1850.8071992799526
time used = 168.3 seconds.


In [ ]:
cv_report3_2 = corss_validate(model_2, train_x_3, train_y_3, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 26405613123.567207 , std = 5280509124.702693
mean mae = 50566.738336206654 , std = 1893.57687424084
mean mape = 2257.6866586389533 , std = 1744.312354197972
time used = 247.41 seconds.


In [ ]:
cv_report3_3 = corss_validate(model_3, train_x_3, train_y_3, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 26000829762.953728 , std = 5497255628.257523
mean mae = 49371.79902709959 , std = 1069.045141992477
mean mape = 2431.0144484773605 , std = 1819.460503819466
time used = 249.46 seconds.


In [ ]:
cv_report3_4 = corss_validate(model_4, train_x_3, train_y_3, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 26072316666.857414 , std = 5298715399.972267
mean mae = 50980.839739928866 , std = 1451.68210990505
mean mape = 2429.6716997611925 , std = 1827.719004040811
time used = 254.62 seconds.


In [ ]:
cv_report3_5 = corss_validate(model_5, train_x_3, train_y_3, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 25999145892.45382 , std = 5660935750.910362
mean mae = 50042.74242197814 , std = 2243.116130663929
mean mape = 2528.9797125371856 , std = 1871.5705890491975
time used = 252.69 seconds.


In [ ]:
cv_report3_6 = corss_validate(model_6, train_x_3, train_y_3, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 26287558358.9504 , std = 5543280881.040758
mean mae = 50619.65134607697 , std = 2581.9505302324455
mean mape = 2363.36738813209 , std = 1736.1604423935685
time used = 248.03 seconds.


In [ ]:
cv_report3_7 = corss_validate(model_7, train_x_3, train_y_3, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 25906523732.538704 , std = 5329145547.329462
mean mae = 49556.45238529069 , std = 1649.2167098332689
mean mape = 2375.8787747398837 , std = 1880.8040502575693
time used = 248.14 seconds.


In [ ]:
cv_report3_8 = corss_validate(model_8, train_x_3, train_y_3, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 26242771174.66324 , std = 5561936796.272775
mean mae = 50453.374474763834 , std = 1809.6816444152987
mean mape = 2392.423867395807 , std = 1809.0866789444535
time used = 160.53 seconds.


In [ ]:
cv_report3 = pd.DataFrame(columns = ['mse', 'mae', 'mape'], index = ['model_1', 'model_2', 'model_3', 'model_4',  'model_5',  'model_6',  'model_7', 'model_8'])
cv_report3.iloc[0, :] = [np.mean(cv_report3_1['mse']), np.mean(cv_report3_1['mae']), np.mean(cv_report3_1['mape'])]
cv_report3.iloc[1, :] = [np.mean(cv_report3_2['mse']), np.mean(cv_report3_2['mae']), np.mean(cv_report3_2['mape'])]
cv_report3.iloc[2, :] = [np.mean(cv_report3_3['mse']), np.mean(cv_report3_3['mae']), np.mean(cv_report3_3['mape'])]
cv_report3.iloc[3, :] = [np.mean(cv_report3_4['mse']), np.mean(cv_report3_4['mae']), np.mean(cv_report3_4['mape'])]
cv_report3.iloc[4, :] = [np.mean(cv_report3_5['mse']), np.mean(cv_report3_5['mae']), np.mean(cv_report3_5['mape'])]
cv_report3.iloc[5, :] = [np.mean(cv_report3_6['mse']), np.mean(cv_report3_6['mae']), np.mean(cv_report3_6['mape'])]
cv_report3.iloc[6, :] = [np.mean(cv_report3_7['mse']), np.mean(cv_report3_7['mae']), np.mean(cv_report3_7['mape'])]
cv_report3.iloc[7, :] = [np.mean(cv_report3_8['mse']), np.mean(cv_report3_8['mae']), np.mean(cv_report3_8['mape'])]
cv_report3.to_csv('cv_report3.csv', index = False)
cv_report3

,mse,mae,mape
model_1,2.61183e+10,49693.2,2400.11
model_2,2.64056e+10,50566.7,2257.69
model_3,2.60008e+10,49371.8,2431.01
model_4,2.60723e+10,50980.8,2429.67
model_5,2.59991e+10,50042.7,2528.98
model_6,2.62876e+10,50619.7,2363.37
model_7,2.59065e+10,49556.5,2375.88
model_8,2.62428e+10,50453.4,2392.42


## 對train_4 cross validate 8個模型

In [ ]:
cv_report4_1 = corss_validate(model_1, train_x_4, train_y_4, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 127142466081.20084 , std = 14247747662.335693
mean mae = 130337.62035008287 , std = 1865.026453146733
mean mape = 4536.890667806001 , std = 6215.9953373896105
time used = 124.42 seconds.


In [ ]:
cv_report4_2 = corss_validate(model_2, train_x_4, train_y_4, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 125082357953.10245 , std = 12962397030.289433
mean mae = 133016.6200087425 , std = 5265.292809899064
mean mape = 4474.332936683419 , std = 6271.172480895436
time used = 183.31 seconds.


In [ ]:
cv_report4_3 = corss_validate(model_3, train_x_4, train_y_4, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 124986062259.1705 , std = 14559544246.140915
mean mae = 131160.50610283308 , std = 2707.2668835479335
mean mape = 4577.942263386345 , std = 6147.283950373793
time used = 183.92 seconds.


In [ ]:
cv_report4_4 = corss_validate(model_4, train_x_4, train_y_4, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 123890085954.73618 , std = 14768431350.090212
mean mae = 134999.7876354421 , std = 5870.847439997523
mean mape = 5016.843302095391 , std = 6298.933395977374
time used = 188.14 seconds.


In [ ]:
cv_report4_5 = corss_validate(model_5, train_x_4, train_y_4, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 124218581485.62357 , std = 13174572283.922882
mean mae = 134013.03779767445 , std = 3004.2264205610845
mean mape = 4801.533407960988 , std = 6366.369507619615
time used = 187.62 seconds.


In [ ]:
cv_report4_6 = corss_validate(model_6, train_x_4, train_y_4, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 124604277092.1185 , std = 13466251529.455229
mean mae = 133383.22780681791 , std = 2214.067201389736
mean mape = 4360.714145446039 , std = 5804.927447670674
time used = 185.66 seconds.


In [ ]:
cv_report4_7 = corss_validate(model_7, train_x_4, train_y_4, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 124231437658.98064 , std = 13843205953.705282
mean mae = 134250.58692567697 , std = 3527.7889089982514
mean mape = 4428.61140678801 , std = 6137.914250878897
time used = 185.79 seconds.


In [ ]:
cv_report4_8 = corss_validate(model_8, train_x_4, train_y_4, 5)

1/5 done.
2/5 done.
3/5 done.
4/5 done.
5/5 done.
mean mse = 127210738951.16838 , std = 14519622596.96619
mean mae = 131425.2548872929 , std = 3218.7782753309543
mean mape = 4716.918951836737 , std = 6748.025287117798
time used = 118.4 seconds.


In [ ]:
cv_report4 = pd.DataFrame(columns = ['mse', 'mae', 'mape'], index = ['model_1', 'model_2', 'model_3', 'model_4',  'model_5',  'model_6',  'model_7', 'model_8'])
cv_report4.iloc[0, :] = [np.mean(cv_report4_1['mse']), np.mean(cv_report4_1['mae']), np.mean(cv_report4_1['mape'])]
cv_report4.iloc[1, :] = [np.mean(cv_report4_2['mse']), np.mean(cv_report4_2['mae']), np.mean(cv_report4_2['mape'])]
cv_report4.iloc[2, :] = [np.mean(cv_report4_3['mse']), np.mean(cv_report4_3['mae']), np.mean(cv_report4_3['mape'])]
cv_report4.iloc[3, :] = [np.mean(cv_report4_4['mse']), np.mean(cv_report4_4['mae']), np.mean(cv_report4_4['mape'])]
cv_report4.iloc[4, :] = [np.mean(cv_report4_5['mse']), np.mean(cv_report4_5['mae']), np.mean(cv_report4_5['mape'])]
cv_report4.iloc[5, :] = [np.mean(cv_report4_6['mse']), np.mean(cv_report4_6['mae']), np.mean(cv_report4_6['mape'])]
cv_report4.iloc[6, :] = [np.mean(cv_report4_7['mse']), np.mean(cv_report4_7['mae']), np.mean(cv_report4_7['mape'])]
cv_report4.iloc[7, :] = [np.mean(cv_report4_8['mse']), np.mean(cv_report4_8['mae']), np.mean(cv_report4_8['mape'])]
cv_report4.to_csv('cv_report4.csv', index = False)
cv_report4

,mse,mae,mape
model_1,1.27142e+11,130338,4536.89
model_2,1.25082e+11,133017,4474.33
model_3,1.24986e+11,131161,4577.94
model_4,1.2389e+11,135000,5016.84
model_5,1.24219e+11,134013,4801.53
model_6,1.24604e+11,133383,4360.71
model_7,1.24231e+11,134251,4428.61
model_8,1.27211e+11,131425,4716.92


In [ ]:
cv_report3 = pd.DataFrame(columns = ['mse', 'mae', 'mape'], index = ['model_1', 'model_2', 'model_3', 'model_4',  'model_5',  'model_6',  'model_7', 'model_8'])
cv_report3.iloc[0, :] = [2.61183e+10,	49693.2,	2400.11]
cv_report3.iloc[1, :] = [2.64056e+10,	50566.7,	2257.69]
cv_report3.iloc[2, :] = [2.60008e+10,	49371.8,	2431.01]
cv_report3.iloc[3, :] = [2.60723e+10,	50980.8,	2429.67]
cv_report3.iloc[4, :] = [2.59991e+10,	50042.7,	2528.98]
cv_report3.iloc[5, :] = [2.62876e+10,	50619.7,	2363.37]
cv_report3.iloc[6, :] = [2.59065e+10,	49556.5,	2375.88]
cv_report3.iloc[7, :] = [2.62428e+10,	50453.4,	2392.42]
cv_report3.to_csv('cv_report3.csv', index = False)
cv_report3

,mse,mae,mape
model_1,2.61183e+10,49693.2,2400.11
model_2,2.64056e+10,50566.7,2257.69
model_3,2.60008e+10,49371.8,2431.01
model_4,2.60723e+10,50980.8,2429.67
model_5,2.59991e+10,50042.7,2528.98
model_6,2.62876e+10,50619.7,2363.37
model_7,2.59065e+10,49556.5,2375.88
model_8,2.62428e+10,50453.4,2392.42


## 每類資料在其表現最好的模型下的test data結果


1. train_x_1: model_3
2. train_x_2: model_2
3. train_x_3: model_7
4. train_x_4: model_7

